In [7]:
import os
import pickle
import random
import time

import PIL
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from tensorflow.keras import Input, Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, ReLU, Reshape, UpSampling2D, Conv2D, Activation, \
    concatenate, Flatten, Lambda, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot as plt


import tensorflow as tf
    
   
def load_class_ids(class_info_file_path):
    """
    Load class ids from class_info.pickle file
    """
    with open(class_info_file_path, 'rb') as f:
        class_ids = pickle.load(f, encoding='latin1')
        return class_ids


def load_embeddings(embeddings_file_path):
    """
    훈련된 텍스트 임베딩을 불러옴
    """
    with open(embeddings_file_path, 'rb') as f:
        embeddings = pickle.load(f, encoding='latin1')
        embeddings = np.array(embeddings)
        print('embeddings: ', embeddings.shape)
    return embeddings

def load_filenames(filenames_file_path):
    """
    Load filenames.pickle file and return a list of all file names
    """
    with open(filenames_file_path, 'rb') as f:
        filenames = pickle.load(f, encoding='latin1')
    return filenames

def load_bounding_boxes(dataset_dir):
    """
    이미지와 그에 상응하는 바운딩 박스를 짝지어 딕셔너리로 만들어 출력
    """
    # 바운딩 박스 전체 경로
    bounding_boxes_path = os.path.join(dataset_dir, 'list_bbox_celeba_pure_del.csv')
    file_paths_path = os.path.join(dataset_dir, 'list_filenames.csv')

    # bounding_boxes.txt 와 images.txt 파일을 읽어옴
    df_bounding_boxes = pd.read_csv(bounding_boxes_path, header=None).astype(int)
    df_file_names = pd.read_csv(file_paths_path, header=None)

    # 전체 이미지 파일 명이 순서대로 적힌 리스트를 만듬
    file_names = df_file_names[0].tolist() 

    # 파일 이름에 대응하는 바운딩 박스가 들어갈 딕셔너리를 만듬 (딕셔너리는 크기를 임의로 증가시킬수 있으므로 초기 사이즈는 아무렇게나)
    filename_boundingbox_dict = {}

    # 이미지 파일과 그에 해당하는 바운딩 박스를 딕셔너리로 만듬 (key = 이미지 파일 이름)
    for i in range(0, len(file_names)):
        # Get the bounding box
        bounding_box = df_bounding_boxes.iloc[i][:].tolist()
        key = file_names[i][:-4] + '.jpg'
        filename_boundingbox_dict[key] = bounding_box

    return filename_boundingbox_dict
'''
새 이미지가 크롭핑 되어있지 않기 크롭하기 위한 바운딩 박스 좌표 값이 파일에 주어지며,
그 파일을 토대로 이미지를 크로핑 한 후,
크로핑된 모든 이미지를 지정한 이미지 크기 (image_size) 값으로 바꾼다
'''
def get_img(img_path, bbox, image_size):
    """
    Load and resize image
    """
    img = Image.open(img_path).convert('RGB')
    width, height = img.size
    if bbox is not None:
        pass
    '''
        R = int(np.maximum(bbox[2], bbox[3]) * 0.75)
        center_x = int((2 * bbox[0] + bbox[2]) / 2)
        center_y = int((2 * bbox[1] + bbox[3]) / 2)
        y1 = np.maximum(0, center_y - R)
        y2 = np.minimum(height, center_y + R)
        x1 = np.maximum(0, center_x - R)
        x2 = np.minimum(width, center_x + R)
        img = img.crop([x1, y1, x2, y2])
    '''
    img = img.resize(image_size, PIL.Image.BILINEAR)
    return img


def load_dataset(filenames_file_path, cub_dataset_dir, embeddings_file_path, image_size):
    """
    Load dataset
    """
    filenames = load_filenames(filenames_file_path)
    '''
    class_ids = load_class_ids(class_info_file_path)
    '''
    bounding_boxes = load_bounding_boxes(cub_dataset_dir)
    all_embeddings = load_embeddings(embeddings_file_path)

    X, y, embeddings = [], [], []

    print("Embeddings shape:", all_embeddings.shape)

    # 각 이미지에 해당하는 바운딩 박스 딕셔너리를 추출하여 get_img 함수로 크로핑되고 같은 크기로 바뀐 이미지를 
    for index, filename in enumerate(filenames):
        bounding_box = bounding_boxes[filename]

        try:
            # Load images
            img_name = '{0}/images/{1}'.format(cub_dataset_dir, filename)
            img = get_img(img_name, bounding_box, image_size)
            '''
            all_embeddings1 = all_embeddings[index, :, :]

            embedding_ix = random.randint(0, all_embeddings1.shape[0] - 1)
            '''
            embedding = all_embeddings[index, :]
            # X = 정제한 이미지 리스트
            X.append(np.array(img))
            '''
            # y = 정제한 이미지 인덱스
            y.append(class_ids[index])
            '''
            # embeddings = 
            embeddings.append(embedding)
        except Exception as e:
            print(e)

    X = np.array(X)
    y = np.array(y)
    embeddings = np.array(embeddings)
    return X, embeddings


def generate_c(x):
    mean = x[:, :128]
    log_sigma = x[:, 128:]
    stddev = K.exp(log_sigma)
    epsilon = K.random_normal(shape=K.constant((mean.shape[1],), dtype='int32'))
    c = stddev * epsilon + mean
    return c


def build_ca_model():
    """
    (1024,)의 텍스트 인코더 신경망의 출력을 입력으로 받고 (256,) 의 텐서를 출력
    """
    input_layer = Input(shape=(1024,))
    x = Dense(256)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    model = Model(inputs=[input_layer], outputs=[x])
    return model


def build_embedding_compressor_model():
    """
    입력 속성값 (40,) 을 (128,) 의 벡터로 확장하는 네트워크
    """
    input_layer = Input(shape=(40,))
    x = Dense(10)(input_layer)
    x = ReLU()(x)

    model = Model(inputs=[input_layer], outputs=[x])
    return model


def build_stage1_generator():
    """
    Stage-I 의 generator 
    *** 이 신경망 안에 CA 신경망과 생성기 신경망이 들어가 있다!!!! ***
    그러므로, 입력으로 텍스트 임베딩 출력 (1024,)과 잡음 변수(100,) 을 받는다
    """
    '''
    input_layer = Input(shape=(1024,))
    x = Dense(256)(input_layer)
    mean_logsigma = LeakyReLU(alpha=0.2)(x)

    c = Lambda(generate_c)(mean_logsigma)

    input_layer2 = Input(shape=(100,))
    gen_input = Concatenate(axis=1)([c, input_layer2])
    '''
    # 텍스트 조건부 변수를 잡음 변수와 접합(concatenation) -> cGAN

    input_layer = Input(shape=(40,))
    x = Dense(10)(input_layer)
    mean_logsigma = LeakyReLU(alpha=0.2)(x)
    
    x = Dense(128 * 8 * 4 * 4, use_bias=False)(input_layer)
    x = ReLU()(x)

    x = Reshape((4, 4, 128 * 8), input_shape=(128 * 8 * 4 * 4,))(x)
    x = Dropout(0.0)(x)
    
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(512, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(256, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(3, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = Activation(activation='tanh')(x)

    stage1_gen = Model(inputs=input_layer, outputs=[x, mean_logsigma])
    '''
    stage - I gen 은 입력된 문장의 임베딩을 바탕으로 (+잡음 변수) 이미지를 생성 함 
    '''
    return stage1_gen


def build_stage1_discriminator():
    """
    Create a model which takes two inputs
    1. One from above network
    2. One from the embedding layer
    3. Concatenate along the axis dimension and feed it to the last module which produces final logits
    """
    input_layer = Input(shape=(64, 64, 3))

    x = Conv2D(64, (4, 4),
               padding='same', strides=2,
               input_shape=(64, 64, 3), use_bias=False)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(128, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)

    x = Conv2D(256, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)

    x = Conv2D(512, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)

    '''
    실제 이미지에 해당하는 압축된 임베딩을 입력
    '''
    input_layer2 = Input(shape=(4, 4, 10))

    '''
    입력 이미지와 압축 텍스트 임베딩을 합침
    '''
    merged_input = concatenate([x, input_layer2])

    x2 = Conv2D(64 * 8, kernel_size=1,
                padding="same", strides=1)(merged_input)
    x2 = BatchNormalization()(x2)
    x2 = LeakyReLU(alpha=0.2)(x2)
    x2 = Flatten()(x2)
    x2 = Dense(1)(x2)
    x2 = Activation('sigmoid')(x2)

    stage1_dis = Model(inputs=[input_layer, input_layer2], outputs=[x2])
    '''
    출력은 입력 이미지가 진짜인지 가짜인지에 관한 확률(sigmoid)을 출력
    '''
    return stage1_dis


def build_adversarial_model(gen_model, dis_model):
    input_layer = Input(shape=(40,))
    input_layer3 = Input(shape=(4, 4, 10))

    x, mean_logsigma = gen_model(input_layer)

    dis_model.trainable = False
    valid = dis_model([x, input_layer3])

    model = Model(inputs=[input_layer, input_layer3], outputs=[valid, mean_logsigma])
    return model


def KL_loss(y_true, y_pred):
    mean = y_pred[:, :128]
    logsigma = y_pred[:, :128]
    loss = -logsigma + .5 * (-1 + K.exp(2. * logsigma) + K.square(mean))
    loss = K.mean(loss)
    return loss


def custom_generator_loss(y_true, y_pred):
    # Calculate binary cross entropy loss
    return K.binary_crossentropy(y_true, y_pred)

'''
def save_rgb_img(img, path):
    """
    Save an rgb image
    """
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title("Image")

    plt.savefig(path)
    plt.close()
'''
def save_rgb_img(img_gen, img_real, path):
    """
    Save an rgb image
    """
    fig = plt.figure()
    ax = fig.add_subplot(1, 2, 1)
    ax.imshow(img_gen)
    ax.axis("off")
    ax.set_title("Image_gen")
    
    ax = fig.add_subplot(1, 2, 2)
    ax.imshow(img_real)
    ax.axis("off")
    ax.set_title("Image_real")
    
    plt.savefig(path)
    plt.close()

def write_log(callback, name, loss, batch_no):
    """
    Write training summary to TensorBoard
    """
    with callback.as_default():
          tf.summary.scalar(name, loss, batch_no)
          callback.flush()








'''
Stage - I stackGAN 훈련
'''
if __name__ == '__main__':
    '''
    하이퍼파라미터(불변 파라미터) 지정
    '''
    data_dir = "/home/csle/Desktop/CelebA_dataset_reduce_cropped"
    train_dir = data_dir + "/train"
    test_dir = data_dir + "/test"
    image_size = 64
    batch_size = 128
    z_dim = 100
    stage1_generator_lr = 0.0002
    stage1_discriminator_lr = 0.0002
    stage1_lr_decay_step = 600
    epochs = 1500
    condition_dim = 10

    embeddings_file_path_train = train_dir + "/attr_(embeddings).pickle"
    embeddings_file_path_test = test_dir + "/attr_(embeddings).pickle"

    filenames_file_path_train = train_dir + "/filenames.pickle"
    filenames_file_path_test = test_dir + "/filenames.pickle"

    '''
    class_info_file_path_train = train_dir + "/class_info.pickle"
    class_info_file_path_test = test_dir + "/class_info.pickle"
    '''

    cub_dataset_dir = data_dir + "/img_align_celeba"

    '''
    optimizer 정의
    '''
    dis_optimizer = Adam(lr=stage1_discriminator_lr, beta_1=0.5, beta_2=0.999)
    gen_optimizer = Adam(lr=stage1_generator_lr, beta_1=0.5, beta_2=0.999)

    """"
    dataset 로드하기
    """
    X_train, embeddings_train = load_dataset(filenames_file_path=filenames_file_path_train,
                                                      cub_dataset_dir=cub_dataset_dir,
                                                      embeddings_file_path=embeddings_file_path_train,
                                                      image_size=(64, 64))

    X_test, embeddings_test = load_dataset(filenames_file_path=filenames_file_path_test,
                                                   cub_dataset_dir=cub_dataset_dir,
                                                   embeddings_file_path=embeddings_file_path_test,
                                                   image_size=(64, 64))
    
    print('finish')

embeddings:  (14261, 40)
Embeddings shape: (14261, 40)
embeddings:  (3565, 40)
Embeddings shape: (3565, 40)
finish


In [9]:
#with tf.device('/gpu:0'): 

import tensorflow as tf 

config = tf.compat.v1.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.6 
session = tf.compat.v1.Session(config=config)

    
"""
신경망 빌드 & compile
"""

ca_model = build_ca_model()
ca_model.compile(loss="binary_crossentropy", optimizer="adam")

stage1_dis = build_stage1_discriminator()
stage1_dis.compile(loss='binary_crossentropy', optimizer=dis_optimizer)

stage1_gen = build_stage1_generator()
stage1_gen.compile(loss="mse", optimizer=gen_optimizer)

embedding_compressor_model = build_embedding_compressor_model()
embedding_compressor_model.compile(loss="binary_crossentropy", optimizer="adam")

'''
stage-I GAN 빌드& 컴파일
이때, stage-I 의 discriminator 는 훈련시키지 않고 stage-I generator 의 가중치만 업데이트
'''
adversarial_model = build_adversarial_model(gen_model=stage1_gen, dis_model=stage1_dis)
adversarial_model.compile(loss=['binary_crossentropy', KL_loss], loss_weights=[1, 2.0],
                          optimizer=gen_optimizer, metrics=None)

"""
tensorboard = TensorBoard(log_dir="logs/".format(time.time()))
tensorboard.set_model(stage1_gen)
tensorboard.set_model(stage1_dis)
'''
tensorboard.set_model(ca_model)
'''
tensorboard.set_model(embedding_compressor_model)
"""
# Generate an array containing real and fake values
# Apply label smoothing as well
real_labels = np.ones((batch_size, 1), dtype=float) * 0.9
fake_labels = np.zeros((batch_size, 1), dtype=float) * 0.1
'''
매 epoch 마다 아래를 반복함
'''
for epoch in range(epochs):
    print("========================================")
    print("Epoch is:", epoch)
    print("Number of batches", int(X_train.shape[0] / batch_size))

    gen_losses = []
    dis_losses = []

    # Load data and train model
    number_of_batches = int(X_train.shape[0] / batch_size)
    for index in range(number_of_batches):
        print("Batch:{}".format(index+1))
        '''
        모델에 입력으로 들어갈 이미지와 텍스트 임베딩을 받아옴 (각 텍스트 임베딩은 각 이미지에 대응 됨)
        '''
        # 원래는 CA 의 출력이 stage-I 으로 들어가야 하지만 gen 안에 CA 가 있음
        z_noise = np.random.normal(0, 1, size=(batch_size, z_dim))
        # 배치 사이즈만큼 훈련(실제) 이미지를 추출
        image_batch = X_train[index * batch_size:(index + 1) * batch_size]
        # 추출한 이미지에 대응하는 임베딩을 추출
        embedding_batch = embeddings_train[index * batch_size:(index + 1) * batch_size]
        # 이미지들을 정규화하여 값을 작게 만듬
        image_batch = (image_batch - 127.5) / 127.5

        # stage-I 의 gen 에서 텍스트 임베딩을 바탕으로 저 해상도 fake 이미지를 생성
        # 이때, 두 stage 는 랜덤하게 생성한 텍스트 임베딩을 나머지 입력으로 받음
        fake_images, _ = stage1_gen.predict(embedding_batch, verbose=3)

        # stage-I dis 에 들어갈 압축 텍스트 임베딩을 랜덤하게 생성한 텍스트 임베딩 기반으로 생성
        compressed_embedding = embedding_compressor_model.predict_on_batch(embedding_batch)
        compressed_embedding = np.reshape(compressed_embedding, (-1, 1, 1, condition_dim))
        compressed_embedding = np.tile(compressed_embedding, (1, 4, 4, 1))
        '''
        discriminator 에서 입력 이미지가 CNN 을 통과한 결과와 속성값 (40,) 을 concatenate 해 주기 위해서는,
        속성값 (40,) 를 compressor 네트워크를 통해 (128,) 로 확장하고,
        reshape, tile 함수를 이용해 CNN 결과 (4, 4, 512) 와 (4, 4, :) 부분을 맞추어 주어야 한다.
        '''

        """
        stage-I dis 를 훈련함
        """
        # 실제 이미지와 압축 텍스트 임베딩을 입력으로 하고 모든 레이블을 1 (이미지가 진짜라는 의미) 로 하여,
        # dis 가 실제 이미지를 잘 분류 하게끔 훈련
        dis_loss_real = stage1_dis.train_on_batch([image_batch, compressed_embedding],
                                                  np.reshape(real_labels, (batch_size, 1)))
        # gen 이 생성한 가짜 이미지와 압축 텍스트 임베딩을 입력으로 하고 모든 레이블을 0 (이미지가 가짜라는 의미) 로 하여,
        # dis 가 가짜 이미지를 잘 분류 하게끔 훈련
        dis_loss_fake = stage1_dis.train_on_batch([fake_images, compressed_embedding],
                                                  np.reshape(fake_labels, (batch_size, 1)))
        # 실제 이미지와 압축 텍스트 임베딩을 '어긋나게' 입력으로 하고 모든 레이블을 0 (이미지가 가짜라는 의미) 로 하여,
        # dis 가 실제 이미지를 잘 분류 하게끔 훈련
        dis_loss_wrong = stage1_dis.train_on_batch([image_batch[:(batch_size - 1)], compressed_embedding[1:]],
                                                   np.reshape(fake_labels[1:], (batch_size-1, 1)))
        dis_loss_wrong2 = stage1_dis.train_on_batch([image_batch[:(batch_size - 2)], compressed_embedding[2:]],
                                                   np.reshape(fake_labels[2:], (batch_size-2, 1)))
        
        d_loss = 0.5*np.add(dis_loss_wrong2, 0.5 * np.add(dis_loss_real, 0.5 * np.add(dis_loss_wrong, dis_loss_fake)))
        
        print("d_loss_real:{}".format(dis_loss_real))
        print("d_loss_fake:{}".format(dis_loss_fake))
        print("d_loss_wrong:{}".format(dis_loss_wrong))
        print("d_loss_wrong2:{}".format(dis_loss_wrong))
        
        print("d_loss:{}".format(d_loss))

        """
        stage-I GAN 을 훈련함
        이때, stage-I 의 discriminator 는 훈련시키지 않고 stage-I generator 의 가중치만 업데이트
        """
        g_loss = adversarial_model.train_on_batch([embedding_batch, compressed_embedding],[K.ones((batch_size, 1)) * 0.9, K.ones((batch_size, 256)) * 0.9])
        print("g_loss:{}".format(g_loss))

        dis_losses.append(d_loss)
        gen_losses.append(g_loss)

    """
    각 epoch 마다 Tensorboard 에 loss 의 전체 배치 평균값 저장
    """
    writer = tf.summary.create_file_writer("logs/".format(time.time()))
    write_log(writer, 'discriminator_loss', np.mean(dis_losses), epoch)
    write_log(writer, 'generator_loss', np.mean(gen_losses[0]), epoch)
    '''
    tf.summary.scalar('discriminator_loss', np.mean(dis_losses), epoch)
    tf.summary.scalar('generator_loss', np.mean(gen_losses[0]), epoch)

    tf.summary.scalar('discriminator_loss', np.mean(dis_losses))
    tf.summary.scalar('generator_loss', np.mean(gen_losses[0]))
    summary_op = tf.summary.merge()
    summary_writer = tf.summary.FileWriter("logs/".format(time.time()))
    '''    
    # 매 두번의 epoch 마다 이미지 gen & 이미지 저장
    if epoch % 2 == 0:
        # z_noise2 = np.random.uniform(-1, 1, size=(batch_size, z_dim))
        z_noise2 = np.random.normal(0, 1, size=(batch_size, z_dim))
        embedding_batch = embeddings_test[0:batch_size]
        fake_images, _ = stage1_gen.predict_on_batch(embedding_batch)

        # Save images
        '''
        for i, img in enumerate(fake_images[:5]):
            save_rgb_img(img, "results/gen_{}_{}.png".format(epoch, i))
        '''
        
        for i in range(5):
            img_gen = fake_images[i]
            img_real = X_test[i]
            save_rgb_img(img_gen, img_real, "results/gen_{}_{}.png".format(epoch, i))


    if epoch % 50 == 0:
        # Save models
        stage1_gen.save_weights("weights/stage1_gen_epoch_{}.h5".format(epoch))
        stage1_dis.save_weights("weights/stage1_dis_epoch_{}.h5".format(epoch))

'''
이제 훈련된 stage-I 의 generator 와 discriminator 을 얻음 (+ embedding_compressor) 
'''

Epoch is: 0
Number of batches 111
Batch:1


KeyboardInterrupt: 